### Load data & preprocessing

In [65]:
import warnings
warnings.filterwarnings('ignore')
import glob

import os
from copy import deepcopy
    
from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import OneHotEncoder, StandardScaler
import random

base = 'c:/Users/JCY/Dacon/shinhan/'
imp=False
def get_data(base, imp=True, ae=False):
    train_df = pd.read_csv(os.path.join(base, 'data/train.csv'))
    test_df = pd.read_csv(os.path.join(base, 'data/test.csv'))
    train_df_target_removed = train_df.drop(columns=['credit'])
    train_df_target = train_df.iloc[:, -1]

    df = pd.concat([train_df_target_removed, test_df], axis=0)

    def birth2age(x):
        return x*(-1) / 365
    df['age'] = df['DAYS_BIRTH'].apply(birth2age)
    df['skill'] = df['DAYS_EMPLOYED'].apply(birth2age)
    df['month'] = df['begin_month'].apply(lambda x: x * (-1))

    df['income_c'] = ''
    df.loc[df['income_total'] <= 1.575e6, 'income_c'] = 'first'
    df.loc[df['income_total'] <= 2.25e5, 'income_c'] ='second'
    df.loc[df['income_total'] <= 1.575e5, 'income_c'] = 'third'
    df.loc[df['income_total'] <= 1.215e5, 'income_c'] = 'fourth'

    if imp:
        df.loc[df['occyp_type'].isnull() &\
               (df['income_type'] == 'Commercial associate') &\
               (df['gender'] == 'F'), 'occyp_type'] = 'Sales staff'
        df.loc[df['occyp_type'].isnull() &\
               (df['income_type'] == 'Commercial associate') &\
               (df['gender'] == 'F') &\
               (df['income_c'] == 'first'), 'occyp_type'] = 'Managers'
        df.loc[df['occyp_type'].isnull() &\
               (df['income_type'] == 'Commercial associate') &\
               (df['gender'] == 'F') &\
               (df['income_c'] == 'third') &\
               (df['work_phone'] == 1), 'occyp_type'] = 'Core staff'
        df.loc[df['occyp_type'].isnull() &\
               (df['income_type'] == 'Commercial associate') &\
               (df['gender'] == 'M'), 'occyp_type'] = 'Drivers'
        df.loc[df['occyp_type'].isnull() &\
               (df['income_type'] == 'Commercial associate') &\
               (df['gender'] == 'M') &\
               (df['income_c'] == 'first'), 'occyp_type'] = 'Managers'
        df.loc[df['occyp_type'].isnull() &\
               (df['income_type'] == 'Commercial associate') &\
               (df['gender'] == 'M') &\
               (df['income_c'] == 'fourth') &\
               (df['work_phone'] == 0), 'occyp_type'] = 'Laborers'
        df.loc[df['occyp_type'].isnull() &\
               (df['income_type'] == 'Commercial associate') &\
               (df['gender'] == 'M') &\
               (df['income_c'] == 'fourth') &\
               (df['work_phone'] == 1), 'occyp_type'] = 'Drivers'
        df.loc[df['occyp_type'].isnull() &\
               (df['income_type'] == 'Pensioner') &\
               (df['gender'] == 'F') &\
               (df['income_c'] == 'first') &\
               (df['work_phone'] == 0), 'occyp_type'] = 'Core staff'
        df.loc[df['occyp_type'].isnull() &\
               (df['income_type'] == 'State servant'), 'occyp_type'] = 'Core staff'
        df.loc[df['occyp_type'].isnull() &\
               (df['income_type'] == 'State servant') &\
               (df['gender'] == 'F') &\
               (df['income_c'] == 'first') &\
               (df['work_phone'] == 1), 'occyp_type'] = 'Managers'
        df.loc[df['occyp_type'].isnull() &\
               (df['income_type'] == 'State servant') &\
               (df['gender'] == 'F') &\
               (df['income_c'] == 'second') &\
               (df['work_phone'] == 0), 'occyp_type'] = 'Medicine staff'
        df.loc[df['occyp_type'].isnull() &\
               (df['income_type'] == 'State servant') &\
               (df['gender'] == 'M') &\
               (df['income_c'] == 'first') &\
               (df['work_phone'] == 1), 'occyp_type'] = 'High skill tech staff'
        df.loc[df['occyp_type'].isnull() &\
               (df['income_type'] == 'State servant') &\
               (df['gender'] == 'M') &\
               (df['income_c'] == 'fouth') &\
               (df['work_phone'] == 0), 'occyp_type'] = 'Laborers'
        df.loc[df['occyp_type'].isnull() &\
               (df['income_type'] == 'State servant') &\
               (df['gender'] == 'M') &\
               (df['income_c'] == 'third') &\
               (df['work_phone'] == 1), 'occyp_type'] = 'Drivers'
        df.loc[df['occyp_type'].isnull() &\
               (df['income_type'] == 'Working'), 'occyp_type'] = 'Laborers'
        df.loc[df['occyp_type'].isnull() &\
               (df['income_type'] == 'Working') &\
               (df['gender'] == 'F'), 'occyp_type'] = 'Sales staff'
        df.loc[df['occyp_type'].isnull() &\
               (df['income_type'] == 'State servant') &\
               (df['gender'] == 'F') &\
               (df['income_c'] == 'first') &\
               (df['work_phone'] == 1), 'occyp_type'] = 'Managers'
        df.loc[df['occyp_type'].isnull() &\
               (df['income_type'] == 'State servant') &\
               (df['gender'] == 'F') &\
               (df['income_c'] == 'second') &\
               (df['work_phone'] == 1), 'occyp_type'] = 'Core staff'
        df.loc[df['occyp_type'].isnull() &\
               (df['income_type'] == 'Student') &\
               (df['income_c'] == 'fourth'), 'occyp_type'] = 'Laborers'
        df.loc[df['occyp_type'].isnull() &\
               (df['income_type'] == 'Student') &\
               (df['income_c'] == 'second'), 'occyp_type'] = 'Core staff'
        df.loc[df['occyp_type'].isnull() &\
               (df['income_type'] == 'Pensioner'), 'occyp_type'] = 'Laborers'
    else:
        df.drop(columns=['occyp_type'], inplace=True)

    df['DAYS_BIRTH_month']=np.floor((-df['DAYS_BIRTH'])/30)-((np.floor((-df['DAYS_BIRTH'])/30)/12).astype(int)*12)
    df['DAYS_BIRTH_week']=np.floor((-df['DAYS_BIRTH'])/7)-((np.floor((-df['DAYS_BIRTH'])/7)/4).astype(int)*4)


    # DAYS_EMPLOYED
    df['DAYS_EMPLOYED_month']=np.floor((-df['DAYS_EMPLOYED'])/30)-((np.floor((-df['DAYS_EMPLOYED'])/30)/12).astype(int)*12)
    df['DAYS_EMPLOYED_week']=np.floor((-df['DAYS_EMPLOYED'])/7)-((np.floor((-df['DAYS_EMPLOYED'])/7)/4).astype(int)*4)

    # before_EMPLOYED
    df['before_EMPLOYED']=df['DAYS_BIRTH']-df['DAYS_EMPLOYED']
    df['before_EMPLOYED_month']=np.floor((-df['before_EMPLOYED'])/30)-((np.floor((-df['before_EMPLOYED'])/30)/12).astype(int)*12)
    df['before_EMPLOYED_week']=np.floor((-df['before_EMPLOYED'])/7)-((np.floor((-df['before_EMPLOYED'])/7)/4).astype(int)*4)

    df.drop(columns=['begin_month', 'DAYS_EMPLOYED', 'DAYS_BIRTH', 'income_c'], inplace=True)
    
    object_col = []
    for col in df.columns:
        if df[col].dtype == 'object':
            object_col.append(col)
    object_col += ['FLAG_MOBIL', 'work_phone', 'phone', 'email']
    enc = OneHotEncoder()
    enc.fit(df.loc[:,object_col])
    
    onehot_df = pd.DataFrame(enc.transform(df.loc[:,object_col]).toarray(), 
                 columns=enc.get_feature_names(object_col))

    new_df = pd.concat([df.reset_index(drop=True), onehot_df], axis=1)
    new_df.drop(columns=object_col, inplace=True)
    
    train_df = new_df[:26457]
    train_df = pd.concat([train_df, train_df_target], axis=1)  
    test_df = new_df[26457:]

    train = train_df.drop(columns=['index'])
    test = test_df.drop(columns=['index'])
    enc_list = enc.get_feature_names(object_col)
    
    
    
    if ae == True:
        enc_result = pd.read_csv('./data/encode_result.csv')
        train = pd.concat([train.drop(columns=enc_list).reset_index(drop=True),
                           enc_result[:26457].reset_index(drop=True)],
                          axis=1).reset_index(drop=True)
        test = pd.concat([test.drop(columns=enc_list).reset_index(drop=True),
                          enc_result[26457:].reset_index(drop=True)],
                         axis=1).reset_index(drop=True)
    
        print('{} features are reduced to {}-d'.format(len(enc.get_feature_names(object_col)),
                                                       len(enc_result.columns)))
    print(object_col)
    print('Shape: train {} test {} total {}'.format(train.shape, test.shape, new_df.shape))
    
    return train, train_df_target, test, new_df, enc_list

In [72]:
base = 'c:/Users/JCY/Dacon/shinhan/'
imp = True
train, train_df_target, test, new_df, enc_list = get_data(base, imp=imp, ae=True)

53 features are reduced to 11-d
['gender', 'car', 'reality', 'income_type', 'edu_type', 'family_type', 'house_type', 'occyp_type', 'FLAG_MOBIL', 'work_phone', 'phone', 'email']
Shape: train (26457, 25) test (10000, 24) total (36457, 67)


### Laboratory

In [75]:
# remove useless columns
train.drop(columns=['DAYS_BIRTH_month', 'DAYS_BIRTH_week', 'DAYS_EMPLOYED_month', 'DAYS_EMPLOYED_week', 'before_EMPLOYED_month', 'before_EMPLOYED_week'], inplace=True)
# child_num
train.loc[(train['child_num'] == 14) | (train['child_num'] == 19), 'child_num'] = 7

In [84]:
train[train['family_size'] <= train['child_num']]

,child_num,income_total,family_size,age,skill,month,before_EMPLOYED,credit,0,1,2,3,4,5,6,7,8,9,10
5825,1,450000.0,1.0,49.789041,1.857534,16.0,-17495,2.0,-0.339249,-0.445096,0.178719,-0.020478,0.093143,0.588779,-0.389705,1.245940,0.092032,0.548464,0.413944
14900,2,225000.0,1.0,40.482192,6.060274,30.0,-12564,2.0,0.267142,0.295780,-0.156389,0.737057,0.352796,0.260599,-0.240041,0.742637,0.209640,0.673847,-0.390758
16110,1,108000.0,1.0,34.857534,3.101370,48.0,-11591,2.0,-0.529618,0.009984,0.908761,-0.283858,-0.656939,0.697711,0.260746,-1.303836,-0.982026,0.284774,-0.163945
16791,1,126000.0,1.0,33.260274,2.134247,37.0,-11361,2.0,0.176429,0.344385,0.318104,0.682796,0.072126,-0.168015,-0.240041,0.301006,0.173253,0.400061,-0.541707
18879,1,126000.0,1.0,34.945205,5.767123,39.0,-10650,2.0,0.158535,0.691184,0.246428,0.218328,-0.117640,0.314804,0.302576,-0.285899,-0.566556,-0.411189,0.138213
21096,1,157500.0,1.0,44.997260,3.994521,25.0,-14966,2.0,0.698603,0.814141,-0.111799,0.240320,0.692513,0.439140,-0.552052,-0.099019,0.148424,0.925686,-0.274781


In [95]:
# 다자녀
train.loc[(train['family_size'] - train['child_num'] == 2) & (train['child_num']) > 2, 'f_type'] = '다자녀'
# 자녀 x
train.loc[train['child_num'] == 0, 'f_type'] = '자녀없음'
# 어른 x
train[train['child_num'] >= train['family_size'], 'f_type'] = '소년소녀가장'
# 이혼 or 사별
train[(train['family_size'] - train['child_num'] == 1) & (train['child_num'] == 0)] = '이혼및사별'
# 독거
train[(train['family_size'] == 1) & (train['child_num'] == 0)] = '독거'
# 대가족
train[(train['family_size'] - train['child_num'] > 2)] = ''

,child_num,income_total,family_size,age,skill,month,before_EMPLOYED,credit,0,1,2,3,4,5,6,7,8,9,10
8462,7,225000.0,15.0,48.641096,4.627397,7.0,-16065,2.0,-0.190920,0.506633,0.473547,0.694755,0.005138,-0.003600,-0.630494,-0.112354,-0.168066,-0.120861,-0.048121
9021,7,225000.0,15.0,48.641096,4.627397,58.0,-16065,2.0,-0.190920,0.506633,0.473547,0.694755,0.005138,-0.003600,-0.630494,-0.112354,-0.168066,-0.120861,-0.048121
10731,7,112500.0,20.0,30.293151,5.076712,5.0,-9204,2.0,-1.553591,-0.041563,0.695188,-1.813192,-0.975374,0.762945,0.481824,0.427113,-0.959787,0.070116,1.620629
25390,7,225000.0,15.0,48.641096,4.627397,43.0,-16065,2.0,-0.190920,0.506633,0.473547,0.694755,0.005138,-0.003600,-0.630494,-0.112354,-0.168066,-0.120861,-0.048121


In [103]:
train.loc[(train['family_size'] - train['child_num'] == 2) & (train['child_num'] <= 2), :]

,child_num,income_total,family_size,age,skill,month,before_EMPLOYED,credit,0,1,2,3,4,5,6,7,8,9,10
0,0,202500.0,2.0,38.079452,12.901370,6.0,-9190,1.0,-0.236222,0.504844,0.650301,-0.475061,-0.399088,-0.467972,-0.290161,0.350561,0.183876,-0.159120,0.118652
1,1,247500.0,3.0,31.178082,4.219178,5.0,-9840,1.0,-0.606603,0.423316,0.363449,0.460144,-0.364225,0.724852,0.481316,0.788715,-0.424211,-0.282902,0.199268
2,0,450000.0,2.0,52.293151,12.147945,22.0,-14653,2.0,-0.331903,-0.581326,-0.047846,-0.565187,-0.237786,0.113906,-0.198013,1.103971,-0.236996,0.278321,0.872392
3,0,202500.0,2.0,41.336986,5.731507,37.0,-12996,0.0,-0.560860,0.205771,0.433703,-0.068904,-0.561791,-0.000265,0.343870,1.048007,-0.291129,-0.581904,0.635004
4,0,157500.0,2.0,41.197260,5.767123,26.0,-12932,2.0,-0.569055,0.223700,0.459858,-0.309174,-0.572924,-0.096957,0.564388,0.945147,-0.436622,-0.458727,0.763352
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26449,0,90000.0,2.0,28.761644,6.624658,2.0,-8080,1.0,2.014828,0.471032,-0.269090,-1.825347,1.251095,0.405840,-2.289332,0.136662,0.641147,2.176540,1.179991
26451,0,202500.0,2.0,35.153425,2.200000,44.0,-12028,1.0,0.806850,0.466059,-0.116846,1.337436,0.051342,-0.102522,-0.245563,0.015639,0.056884,0.397718,-0.944252
26452,2,225000.0,4.0,33.093151,5.435616,2.0,-10095,1.0,-0.106534,-0.072626,0.361258,-0.131738,-0.032355,-0.068052,-0.537398,0.601244,0.221627,0.445607,0.017607
26454,0,292500.0,2.0,27.621918,5.520548,25.0,-8067,2.0,0.707271,-0.588423,0.948967,-0.212728,0.277019,-0.128983,0.211407,-0.509178,-0.374360,0.665366,-0.078441


### Training

In [41]:
k = 5

skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)
folds=[]
for train_idx, valid_idx in skf.split(train, train['credit']):
    folds.append((train_idx, valid_idx))
    
random.seed(42)
xgb_models = {}
scores = []
for fold in range(k):
    print(f'===================================={fold+1}============================================')
    train_idx, valid_idx = folds[fold]
    X_train, X_valid, y_train, y_valid = train.drop(['credit'],axis=1).iloc[train_idx].values, train.drop(['credit'],axis=1).iloc[valid_idx].values,\
                                         train['credit'][train_idx].values, train['credit'][valid_idx].values 
    xgb = XGBClassifier(
        objective='multi:softmax',
        grow_policy='lossguide',
        
        num_class=3,
        learning_rate=0.01, #0.005
        n_estimators=3000,
        
        max_depth=15,
        min_child_weight=3,
        gamma=0.3,
        subsample=0.9,
        colsample_bytree=0.6,
        reg_alpha=1,
    )
    xgb.fit(X_train, y_train, 
        eval_set=[(X_train, y_train), (X_valid, y_valid)],
        eval_metric='mlogloss',
        early_stopping_rounds=30,
        verbose=100)
    xgb_models[fold]=xgb
    scores.append(xgb.best_score)
    print(f'================================================================================\n\n')
print('-' * 50, 'Result' ,'-' * 50, '\n')
print('Mean val score : {}'.format(np.mean(scores, axis=0), '\n'))
print('_' * 106)

====================================1============================================
[0]	validation_0-mlogloss:1.09282	validation_1-mlogloss:1.09362
[100]	validation_0-mlogloss:0.76241	validation_1-mlogloss:0.83152
[200]	validation_0-mlogloss:0.61575	validation_1-mlogloss:0.74751
[300]	validation_0-mlogloss:0.52980	validation_1-mlogloss:0.71500


KeyboardInterrupt: 

1. learning_rate=0.1
Mean val score : 0.7013356
    
2. learning_rate=0.01
Mean val score : 0.6973156
    
3. learning_rate=0.005
Mean val score : 0.6968796

4. preperly encoded
Mean val score : 0.6857517

5. 
'begin_month', 'DAYS_EMPLOYED', 'DAYS_BIRTH' not removed
Mean val score : 0.6901195

In [30]:
from sklearn.model_selection import GridSearchCV, train_test_split


X = train.drop(['credit'],axis=1)
y = train['credit']

train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.2, random_state=0)

params = {
    'max_depth':[15],
    'min_child_weight':[3],
    'gamma':[0.3],
    'subsample':[0.7],
    'colsample_bytree':[0.4],
    'reg_alpha':[1],
    'reg_lambda':[0.4],
    'grow_policy':['lossguide']
}
gs = GridSearchCV(XGBClassifier(
        objective='multi:softmax',
        num_class=3,
        learning_rate=0.1,
        n_estimators=1500
    ), params, scoring='neg_log_loss', cv=3)

gs.fit(train_X, train_y,
       eval_metric='mlogloss',
       eval_set=[(train_X, train_y), (test_X, test_y)],
       early_stopping_rounds=30,
       verbose=100)
print('Best : {} with {}'.format(gs.best_score_, gs.best_params_))

[0]	validation_0-mlogloss:1.05526	validation_1-mlogloss:1.06110
[82]	validation_0-mlogloss:0.45307	validation_1-mlogloss:0.74188
[0]	validation_0-mlogloss:1.05488	validation_1-mlogloss:1.06094
[81]	validation_0-mlogloss:0.46001	validation_1-mlogloss:0.74283
[0]	validation_0-mlogloss:1.05640	validation_1-mlogloss:1.06148
[82]	validation_0-mlogloss:0.45745	validation_1-mlogloss:0.73519
[0]	validation_0-mlogloss:1.05603	validation_1-mlogloss:1.06140
[85]	validation_0-mlogloss:0.46023	validation_1-mlogloss:0.73922
[0]	validation_0-mlogloss:1.05519	validation_1-mlogloss:1.06098
[84]	validation_0-mlogloss:0.46535	validation_1-mlogloss:0.74062
[0]	validation_0-mlogloss:1.05690	validation_1-mlogloss:1.06198
[84]	validation_0-mlogloss:0.46548	validation_1-mlogloss:0.73584
[0]	validation_0-mlogloss:1.05650	validation_1-mlogloss:1.06176
[85]	validation_0-mlogloss:0.47050	validation_1-mlogloss:0.73830
[0]	validation_0-mlogloss:1.05627	validation_1-mlogloss:1.06178
[86]	validation_0-mlogloss:0.4747

### Submission

In [5]:
submit = pd.read_csv('data/sample_submission.csv')
submit.iloc[:,1:]=0
for fold in range(k):
    submit.iloc[:,1:] += xgb_models[fold].predict_proba(test) / k
    
submit.to_csv('./submission/xgb/submission0.csv', index=False) 
pd.read_csv('./submission/xgb/submission0.csv')

,index,0,1,2
0,26457,0.060614,0.134896,0.804490
1,26458,0.326098,0.247175,0.426727
2,26459,0.056933,0.069057,0.874010
3,26460,0.060049,0.089649,0.850302
4,26461,0.082470,0.159123,0.758407
...,...,...,...,...
9995,36452,0.063126,0.220409,0.716465
9996,36453,0.378548,0.299830,0.321621
9997,36454,0.057095,0.080827,0.862079
9998,36455,0.198014,0.418498,0.383488


### 0
```
xgb = XGBClassifier(
        objective='multi:softmax',
        num_class=3,
        learning_rate=0.005,
        n_estimators=3000,
        
        max_depth=15,
        min_child_weight=3,
        gamma=0.3,
        subsample=0.9,
        colsample_bytree=0.6,
        reg_alpha=1,
    )
```
```
Mean val score : 0.6857517
```